In [5]:
from bs4 import BeautifulSoup as bsoup
import urllib.robotparser
import requests
import pandas as pd
import numpy as np
from pprint import pprint
from selenium import webdriver

# Import GeckoDriverManager module.
from webdriver_manager.firefox import GeckoDriverManager
# Install the GeckoDriverManager to run FireFox web browser.
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())



====== WebDriver manager ======
Current firefox version is 1 cannot be loaded. 
Get LATEST driver version for 1 cannot be loaded. 
Driver [C:\Users\Sam\.wdm\drivers\geckodriver\win64\v0.29.1\geckodriver.exe] found in cache


In [6]:
# formats to scrape, based on Bandcamp URL options
FORMATS = ['all',
           'digital',
           'vinyl',
           'cd',
           'cassette']

# search categories
#   top = best-selling
#   new = new arrivals
#   rec = artist-recommended
SEARCH_CATEGORIES = ['top','new','rec']

# scrape dataframe
scrape_data = pd.DataFrame(columns=['Genre',
                                    'Sub-Genre',
                                    'Release Name', 
                                    'Artist Name', 
                                    'Artist Location',
                                    'Release Date',
                                    'Track Information',
                                    'Fan Popularity Score',
                                    'Album Tags'])

# dictionary entry for track listing
track_info_entry = {
    'Track Title' : '',
    'Track Lyrics' : '',
    'Track Number' : '',
    'Track Duration': ''
}

In [7]:
# scrape bandcamp release page function
def scrape_release_page(URL):
    
    pass

In [11]:
# scrape search page function
def scrape_search_page(URL):

    #driver = webdriver.Firefox(executable_path=r'geckodriver.exe')
    #driver = webdriver.Firefox(executable_path=r'your\path\geckodriver.exe')
    #driver = webdriver.Chrome(executable_path='.\chromedriver.exe')
    driver.get(URL)
    html = driver.page_source

    pprint(html)
    #response = requests.get(URL)
    
    #if response.status_code == 200:
    #    print(URL)
    #    soup = bsoup(response.text, 'lxml')
    #    pprint(soup)

In [9]:
###################### INPUTS ################################

# list of genres to scrape
genres = ['all','rock']

# scrape for subgenres within genres?
scrape_subgenres = True

# dictionary of subgenres to scrape, if scrape_subgenres = True
SUBGENRES = {
                'rock': ['all','indie','prog-rock','post-rock','rock-roll','psychedelic-rock'],
                'metal': [],
                'alternative': [],
                'hip-hop-rap': [],
                'experimental': [],
                'punk': [],
                'folk': [],
                'pop': [],
                'acoustic': [],
                'funk': [],
                'country': [],
                'blues': []
            }

# list of search categories to scrape
search_categories = ['top']

# list of locations to scrape
# location = 0 returns search results for all locations
locations = [0] 

# list of formats to scrape
formats = ['all']

# number of pages to scrape
pages = 10 

###########################################################


In [12]:
# web scraper loop

for genre in genres:
    for category in search_categories:
        for location in locations:
            for formatt in formats:
                for page in range(pages):
                    
                    scrape_URL = 'https://bandcamp.com/?g=' + genre + '&s=' + category + '&p=' + str(page) + '&gn=' + str(location) + '&f=' + formatt
                    scrape_search_page(scrape_URL)
                    break
                    
                    if scrape_subgenres and genre in SUBGENRES:
                        for subgenre in SUBGENRES[genre]:
                            subgenre_scrape_URL = scrape_URL + '&t=' + subgenre
                            scrape_search_page(subgenre_scrape_URL)           

('<html xmlns:og="http://opengraphprotocol.org/schema/" '
 'xmlns:fb="http://www.facebook.com/2008/fbml" class="js has-svg has-cssom '
 'has-hover no-touch has-anim"><head>\n'
 '    <title>Bandcamp</title>\n'
 '    \n'
 '    \n'
 '\n'
 '\n'
 '\n'
 '    <meta name="description" content="\n'
 'Discover amazing music and directly support the artists who make it.\n'
 '">\n'
 '    \n'
 '    \n'
 '    <link rel="apple-touch-icon" sizes="180x180" '
 'href="https://s4.bcbits.com/img/favicon/apple-touch-icon.png">\n'
 '    <link rel="icon" type="image/png" sizes="32x32" '
 'href="https://s4.bcbits.com/img/favicon/favicon-32x32.png">\n'
 '    <link rel="icon" type="image/png" sizes="16x16" '
 'href="https://s4.bcbits.com/img/favicon/favicon-16x16.png">\n'
 '    <link rel="mask-icon" '
 'href="https://s4.bcbits.com/img/favicon/safari-pinned-tab.svg" '
 'color="#1da0c3">\n'
 '\n'
 '\n'
 '<meta name="msapplication-TileColor" content="#603cba">\n'
 '<meta name="theme-color" content="#ffffff">\n'
 ' 

 'L3,3.887 L3,15.989 C3,16.541 3.447,16.988 3.999,16.988 L14.001,16.988 '
 'C14.553,16.988 15,16.541 15,15.989 L15,3.887 L17,3.887 L17,2.887 '
 'L14.001,2.887 Z"></path>\n'
 '    </g>\n'
 '\n'
 '    <!-- <svg viewBox="0 0 16 16" xmlns="http://www.w3.org/2000/svg"></svg> '
 '-->\n'
 '    <g id="ic-edit">\n'
 '        <svg viewBox="0 0 16 16" xmlns="http://www.w3.org/2000/svg">\n'
 '            <path d="m12.02 6.27 1.06-1.07a.56.56 0 0 0 '
 '.17-.41c0-.16-.06-.3-.17-.41l-1.36-1.36a.56.56 0 0 0-.4-.17c-.17 '
 '0-.31.06-.42.17L9.83 4.08l2.19 2.19zm-7.09 7.08L11.4 6.9 9.2 4.7l-6.45 '
 '6.46v2.18h2.18z"></path>\n'
 '        </svg>\n'
 '    </g>\n'
 '\n'
 '    <!-- <svg width="16" height="16" xmlns="http://www.w3.org/2000/svg"> '
 '-->\n'
 '    <g id="ic-tooltip">\n'
 '        <path d="M8 15.06a6.55 6.55 0 0 0 '
 '5.74-3.28c.61-1.04.92-2.16.92-3.38a6.55 6.55 0 0 0-3.28-5.74A6.55 6.55 0 0 0 '
 '8 1.74c-1.2 0-2.33.3-3.36.92-1 .58-1.8 1.38-2.38 2.38a6.52 6.52 0 0 0-.92 '
 '3.36c0 1.2.3 2.33.92 3

 '        <p class="bcdaily-author">Kezia Cochrane</p>\n'
 '    </div>\n'
 '    \n'
 '    <span class="story-type">\n'
 '        \n'
 '        Feature\n'
 '\n'
 '    </span>\n'
 '</a>\n'
 '\n'
 '    \n'
 '        \n'
 '        \n'
 '            \n'
 '        \n'
 '        \n'
 '            \n'
 '        \n'
 '        \n'
 '\n'
 '    \n'
 '\n'
 '\n'
 '\n'
 '\n'
 '<a class="col col-3-15 bcdaily-story story-type-l desktop-batch-2 '
 'desktop-only " '
 'href="https://daily.bandcamp.com/scene-report/gothenburg-underground-scene-report">\n'
 '    \n'
 '    <div class="bcdaily-thumb"><div class="bcdaily-thumb-img" '
 'style="background-image: '
 'url(https://f4.bcbits.com/img/0025761991_23.jpg);"></div></div>\n'
 '    <div class="bcdaily-details">\n'
 '        <h3 class="bcdaily-title">A Guide to the Gothenburg '
 'Underground</h3>\n'
 '        <p class="bcdaily-author">Louis Pattison</p>\n'
 '    </div>\n'
 '    \n'
 '    <span class="story-type">\n'
 '        \n'
 '        List\n'
 '\n'
 ' 

 'C10.926,3.78818899 11,3.65518899 11,3.50018899 C11,3.34518899 '
 '10.926,3.21218899 10.815,3.12018899 Z"></path>\n'
 '        <path id="bc-daily-expand" '
 'd="M9,4H6V1c0-0.6-0.4-1-1-1S4,0.4,4,1v3H1C0.4,4,0,4.4,0,5c0,0.6,0.4,1,1,1h3v3c0,0.6,0.4,1,1,1 '
 's1-0.4,1-1V6h3c0.6,0,1-0.4,1-1C10,4.4,9.6,4,9,4z"></path>\n'
 '        <path id="bc-daily-arr" d="M10,3.50018899 C10,3.34518899 '
 '9.926,3.21218899 9.815,3.12018899 C9.809,3.11518899 9.808,3.10618899 '
 '9.802,3.10118899 L5.58,0.0611889922 C5.427,-0.0388110078 5.221,-0.0138110078 '
 '5.097,0.124188992 C4.973,0.260188992 4.967,0.467188992 5.083,0.610188992 '
 'L6.863,3.00018899 L0.5,3.00018899 C0.224,3.00018899 0,3.22418899 '
 '0,3.50018899 C0,3.77618899 0.224,4.00018899 0.5,4.00018899 L6.863,4.00018899 '
 'L5.083,6.39018899 C4.967,6.53318899 4.973,6.74018899 5.097,6.87618899 '
 'C5.221,7.01418899 5.427,7.03918899 5.58,6.93918899 L9.802,3.89918899 '
 'C9.808,3.89418899 9.809,3.88518899 9.815,3.88018899 C9.926,3.78818899 '
 '10,3.6551

 'id="→"></path>\n'
 '            </g>\n'
 '        </g>\n'
 '    </g>\n'
 '    <g id="has-app">\n'
 '        <path class="has-app-phone" d="M18 56a2 2 0 002-2c0-1.1-.9-2-2-2s-2 '
 '.9-2 2 .9 2 2 2zM29 0H7C3.1 0 0 3.1 0 7v48c0 3.9 3.1 7 7 7h22c3.9 0 7-3.1 '
 '7-7V7c0-3.9-3.1-7-7-7zm5 55c0 2.8-2.2 5-5 5H7c-2.8 '
 '0-5-2.2-5-5v-6h32v6zm0-8H2V12h32v35zm0-37H2V7c0-2.8 2.2-5 5-5h22c2.8 0 5 2.2 '
 '5 5v3z"></path>\n'
 '        <path class="has-app-confirm" d="M18 18c-6.4 0-11.5 5.1-11.5 '
 '11.5S11.6 41 18 41s11.5-5.1 11.5-11.5S24.4 18 18 18zm5.3 9.6l-5.8 '
 '5.8c-.3.3-.7.4-1.1.4-.4 0-.8-.1-1.1-.4l-2.6-2.6c-.6-.6-.6-1.5 0-2.1s1.5-.6 '
 '2.1 0l1.6 1.6 4.8-4.8c.6-.6 1.5-.6 2.1 0 .6.6.6 1.6 0 2.1z"></path>\n'
 '    </g>\n'
 '    <g id="no-app">\n'
 '        <path class="no-app-phone" d="M18 56a2 2 0 002-2c0-1.1-.9-2-2-2s-2 '
 '.9-2 2 .9 2 2 2zM29 0H7C3.1 0 0 3.1 0 7v48c0 3.9 3.1 7 7 7h22c3.9 0 7-3.1 '
 '7-7V7c0-3.9-3.1-7-7-7zm5 55c0 2.8-2.2 5-5 5H7c-2.8 '
 '0-5-2.2-5-5v-6h32v6zm0-8H2V12h32v35zm

 '    </div>\n'
 '</div>\n'
 '\n'
 '<div class="bcweekly-info bd-section">\n'
 '    <div class="row">\n'
 '        <div class="bcweekly-info-inner">\n'
 '            <div class="bcweekly-player">\n'
 '    <div class="col">\n'
 '        <a href="#" class="play-btn" style="background-color: #b85587"><span '
 'class="icon"></span></a>\n'
 '    </div>\n'
 '    <div class="col">\n'
 '        \n'
 '            <h2 class="bcweekly-title">The Hip-Hop Show <em>August 13, '
 '2021</em></h2>\n'
 '            <p class="bcweekly-desc">MIKE joins the show to discuss his '
 'recent release, "Disco!". Featuring tunes by Quadry, Reaper Mook, and '
 'more.</p>\n'
 '            <p class="bcweekly-credits">Hosted by <a '
 'href="https://stoneycreation.bandcamp.com/" _blank"="">Stoney Creation</a>. '
 'Illustration of <a '
 'href="https://mikelikesrap.bandcamp.com/album/disco">MIKE</a> by <a '
 'href="https://mckayfelt.site/" _blank"="">McKay Felt</a>.</p>\n'
 '            <p class="bcweekly-links-1"><a hr

 '    <div class="bcdaily-details">\n'
 '        <h3 class="bcdaily-title">Gas Tanks &amp; Synesthesia: The Free Jazz '
 'of Germán Bringas</h3>\n'
 '        <p class="bcdaily-author">Andy Beta</p>\n'
 '    </div>\n'
 '    \n'
 '    <span class="story-type">\n'
 '        \n'
 '        Feature\n'
 '\n'
 '    </span>\n'
 '</a>\n'
 '\n'
 '    \n'
 '        \n'
 '        \n'
 '            \n'
 '        \n'
 '        \n'
 '            \n'
 '        \n'
 '        \n'
 '\n'
 '    \n'
 '\n'
 '\n'
 '\n'
 '\n'
 '<a class="col col-3-15 bcdaily-story story-type-l desktop-batch-3 '
 'desktop-only " '
 'href="https://daily.bandcamp.com/scene-report/a-guide-to-newcastles-improvisational-scene">\n'
 '    \n'
 '    <div class="bcdaily-thumb"><div class="bcdaily-thumb-img" '
 'style="background-image: '
 'url(https://f4.bcbits.com/img/0025687953_23.jpg);"></div></div>\n'
 '    <div class="bcdaily-details">\n'
 '        <h3 class="bcdaily-title">A Guide to Newcastle’s Improvisational '
 'Scene</h3>\n'
 '